## 1. Perkenalan

DS=Suma Bimantoro Andru

## 2.- Import Libraries

In [85]:
# Import Libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler,OrdinalEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import preprocessing
import plotly.express as px
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras import Model 
import tensorflow as tf
from tensorflow import keras
import pickle
import requests
import json
import requests

## 3 - Data Loading

In [86]:
#import data train
data1=pd.read_csv('Train-1542865627584.csv') #label #done
data2=pd.read_csv('Train_Beneficiarydata-1542865627584.csv')
data3=pd.read_csv('Train_Inpatientdata-1542865627584.csv')
data4=pd.read_csv('Train_Outpatientdata-1542865627584.csv')

In [87]:
#import data test
datates1=pd.read_csv('Test-1542969243754.csv') 
datates2=pd.read_csv('Test_Beneficiarydata-1542969243754.csv')
datates3=pd.read_csv('Test_Inpatientdata-1542969243754.csv')
datates4=pd.read_csv('Test_Outpatientdata-1542969243754.csv')

## EDA


## Feature Selection

In [88]:
#Pilih Feature yang berkorelasi dari Beneficiarydata
data2new=data2[['Gender','RenalDiseaseIndicator','NoOfMonths_PartACov','NoOfMonths_PartBCov','ChronicCond_Alzheimer','ChronicCond_Heartfailure','ChronicCond_KidneyDisease','ChronicCond_Cancer','ChronicCond_ObstrPulmonary','ChronicCond_Depression','ChronicCond_Diabetes','ChronicCond_IschemicHeart','ChronicCond_Osteoporasis','ChronicCond_rheumatoidarthritis','ChronicCond_stroke','IPAnnualReimbursementAmt','IPAnnualDeductibleAmt','OPAnnualReimbursementAmt','OPAnnualDeductibleAmt']]
data2tesnew=datates2[['Gender','RenalDiseaseIndicator','NoOfMonths_PartACov','NoOfMonths_PartBCov','ChronicCond_Alzheimer','ChronicCond_Heartfailure','ChronicCond_KidneyDisease','ChronicCond_Cancer','ChronicCond_ObstrPulmonary','ChronicCond_Depression','ChronicCond_Diabetes','ChronicCond_IschemicHeart','ChronicCond_Osteoporasis','ChronicCond_rheumatoidarthritis','ChronicCond_stroke','IPAnnualReimbursementAmt','IPAnnualDeductibleAmt','OPAnnualReimbursementAmt','OPAnnualDeductibleAmt']]

In [89]:
#Pilih Feature yang berkorelasi dari Inpatientdata
data3new=data3[['Provider','InscClaimAmtReimbursed','DeductibleAmtPaid']]
data3tesnew=datates3[['Provider','InscClaimAmtReimbursed','DeductibleAmtPaid']]

In [90]:
#Pilih Feature yang berkorelasi dari Outpatientdata
data4new=data4[['Provider','InscClaimAmtReimbursed','DeductibleAmtPaid']]
data4tesnew=datates4[['Provider','InscClaimAmtReimbursed','DeductibleAmtPaid']]

In [91]:
#cek data
data3new.head(3)

,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid
0,PRV55912,26000,1068.0
1,PRV55907,5000,1068.0
2,PRV56046,5000,1068.0


In [92]:
#cek data
data4new.head(3)

,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid
0,PRV56011,30,0
1,PRV57610,80,0
2,PRV57595,10,0


In [93]:
#penggabungan data untuk proses pembuatan model
final_train = pd.concat([data1, data2new,data3new,data4new], axis=1)
final_test = pd.concat([datates1, data2tesnew,data3tesnew,data4tesnew], axis=1)

In [94]:
final_test

,Provider,Gender,RenalDiseaseIndicator,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,...,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid
0,PRV51002,1.0,0,12.0,12.0,1.0,2.0,1.0,2.0,2.0,...,36000.0,3204.0,60.0,70.0,PRV57070,9000.0,1068.0,PRV55962,30,0
1,PRV51006,1.0,0,12.0,12.0,1.0,1.0,2.0,2.0,2.0,...,0.0,0.0,1490.0,160.0,PRV54750,14000.0,1068.0,PRV56112,30,0
2,PRV51009,2.0,0,12.0,12.0,2.0,1.0,2.0,1.0,1.0,...,0.0,0.0,1170.0,660.0,PRV53758,2000.0,1068.0,PRV56979,200,0
3,PRV51010,2.0,0,12.0,12.0,2.0,1.0,1.0,2.0,2.0,...,5000.0,1068.0,250.0,320.0,PRV55825,16000.0,1068.0,PRV56573,10,0
4,PRV51018,2.0,Y,12.0,12.0,2.0,1.0,1.0,2.0,1.0,...,21260.0,2136.0,120.0,100.0,PRV52338,19000.0,1068.0,PRV56573,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRV55485,70,0
125837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRV55485,10,0
125838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRV55485,30,0
125839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRV55889,30,0


In [95]:
final_train.shape

(517737, 27)

In [96]:
final_test.shape

(125841, 26)

## handling Missing Value

In [97]:
final_train.isnull().sum()

Provider                           512327
PotentialFraud                     512327
Gender                             379181
RenalDiseaseIndicator              379181
NoOfMonths_PartACov                379181
NoOfMonths_PartBCov                379181
ChronicCond_Alzheimer              379181
ChronicCond_Heartfailure           379181
ChronicCond_KidneyDisease          379181
ChronicCond_Cancer                 379181
ChronicCond_ObstrPulmonary         379181
ChronicCond_Depression             379181
ChronicCond_Diabetes               379181
ChronicCond_IschemicHeart          379181
ChronicCond_Osteoporasis           379181
ChronicCond_rheumatoidarthritis    379181
ChronicCond_stroke                 379181
IPAnnualReimbursementAmt           379181
IPAnnualDeductibleAmt              379181
OPAnnualReimbursementAmt           379181
OPAnnualDeductibleAmt              379181
Provider                           477263
InscClaimAmtReimbursed             477263
DeductibleAmtPaid                 

In [98]:
final_test.isnull().sum()

Provider                           124488
Gender                              61873
RenalDiseaseIndicator               61873
NoOfMonths_PartACov                 61873
NoOfMonths_PartBCov                 61873
ChronicCond_Alzheimer               61873
ChronicCond_Heartfailure            61873
ChronicCond_KidneyDisease           61873
ChronicCond_Cancer                  61873
ChronicCond_ObstrPulmonary          61873
ChronicCond_Depression              61873
ChronicCond_Diabetes                61873
ChronicCond_IschemicHeart           61873
ChronicCond_Osteoporasis            61873
ChronicCond_rheumatoidarthritis     61873
ChronicCond_stroke                  61873
IPAnnualReimbursementAmt            61873
IPAnnualDeductibleAmt               61873
OPAnnualReimbursementAmt            61873
OPAnnualDeductibleAmt               61873
Provider                           116290
InscClaimAmtReimbursed             116290
DeductibleAmtPaid                  116486
Provider                          

In [99]:
final_train.dropna(inplace=True)

In [100]:
final_test.dropna(inplace=True)

In [101]:
final_train.isnull().sum()

Provider                           0
PotentialFraud                     0
Gender                             0
RenalDiseaseIndicator              0
NoOfMonths_PartACov                0
NoOfMonths_PartBCov                0
ChronicCond_Alzheimer              0
ChronicCond_Heartfailure           0
ChronicCond_KidneyDisease          0
ChronicCond_Cancer                 0
ChronicCond_ObstrPulmonary         0
ChronicCond_Depression             0
ChronicCond_Diabetes               0
ChronicCond_IschemicHeart          0
ChronicCond_Osteoporasis           0
ChronicCond_rheumatoidarthritis    0
ChronicCond_stroke                 0
IPAnnualReimbursementAmt           0
IPAnnualDeductibleAmt              0
OPAnnualReimbursementAmt           0
OPAnnualDeductibleAmt              0
Provider                           0
InscClaimAmtReimbursed             0
DeductibleAmtPaid                  0
Provider                           0
InscClaimAmtReimbursed             0
DeductibleAmtPaid                  0
d

In [102]:
final_test.isnull().sum()

Provider                           0
Gender                             0
RenalDiseaseIndicator              0
NoOfMonths_PartACov                0
NoOfMonths_PartBCov                0
ChronicCond_Alzheimer              0
ChronicCond_Heartfailure           0
ChronicCond_KidneyDisease          0
ChronicCond_Cancer                 0
ChronicCond_ObstrPulmonary         0
ChronicCond_Depression             0
ChronicCond_Diabetes               0
ChronicCond_IschemicHeart          0
ChronicCond_Osteoporasis           0
ChronicCond_rheumatoidarthritis    0
ChronicCond_stroke                 0
IPAnnualReimbursementAmt           0
IPAnnualDeductibleAmt              0
OPAnnualReimbursementAmt           0
OPAnnualDeductibleAmt              0
Provider                           0
InscClaimAmtReimbursed             0
DeductibleAmtPaid                  0
Provider                           0
InscClaimAmtReimbursed             0
DeductibleAmtPaid                  0
dtype: int64

In [103]:
final_train.shape

(5290, 27)

In [104]:
final_test.shape

(1337, 26)

Peneliti Memutuskan untuk tidak mengambil `miss value` untuk pembuatan model prediksi, dan terlihat perbedaan shape dikarenakan label untuk tabel test tidak ada.